# Neural Network example

Code Source: [Youtube Video](https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3)


In [5]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc
import torch.nn.functional as F # All functions that dont have any parameters
from torch.utils.data import DataLoader # Gives easier dataset management and creates mini batches
import torchvision.datasets as datasets # has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

In [7]:
# create Fully Connected Network

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [8]:
model = NN(784, 10)
model

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [10]:
x = torch.rand(64, 784)
x

tensor([[0.4936, 0.3882, 0.7754,  ..., 0.9967, 0.7277, 0.5109],
        [0.3345, 0.8422, 0.0608,  ..., 0.2909, 0.1578, 0.5992],
        [0.6401, 0.4978, 0.1898,  ..., 0.8452, 0.7062, 0.6356],
        ...,
        [0.7772, 0.4562, 0.2310,  ..., 0.7136, 0.1919, 0.4753],
        [0.0156, 0.1696, 0.9584,  ..., 0.4892, 0.7709, 0.8500],
        [0.7112, 0.5734, 0.5988,  ..., 0.2066, 0.5188, 0.1146]])

In [11]:
model(x).shape

torch.Size([64, 10])

In [12]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [31]:
# Hyperparameters

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size=64
num_epochs = 3

In [14]:
# Load Data

train_dataset = datasets.MNIST(root="dataset/", train = True, transform = transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", train = False, transform = transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


100%|██████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 16185336.65it/s]


Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 412639.51it/s]


Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 7326194.94it/s]


Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4540164.15it/s]

Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw

In [15]:
# Initialize Network

model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [16]:
model

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [17]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
criterion

CrossEntropyLoss()

In [19]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [32]:
# Train Network

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        

In [33]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            __, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
        print(f"Got ({num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f})")
        acc = num_correct / num_samples *100
    model.train()
    return acc

In [29]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got (56185 / 60000 with accuracy 93.64)


tensor(93.6417)

In [30]:
check_accuracy(test_loader, model)

Checking accuracy on test data
Got (9344 / 10000 with accuracy 93.44)


tensor(93.4400)

In [ ]:
# now i rerun everything but with num_epochs=3

In [34]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got (57993 / 60000 with accuracy 96.66)


tensor(96.6550)

In [35]:
check_accuracy(test_loader, model)

Checking accuracy on test data
Got (9597 / 10000 with accuracy 95.97)


tensor(95.9700)

# CNN

Code Source: [Youtube Video](https://www.youtube.com/watch?v=wnK3uWv_WkU&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=4)


In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_class):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d()